In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
def articles():
    with open('page_revisions_text', 'rb') as text_file:
        pending_article_data = b''
        while True:
            data = text_file.read(1024 * 1024)
            if len(data) == 0:
                break

            articles = data.split(b'\0')
            articles[0] = pending_article_data + articles[0]
            for index, article in enumerate(articles):
                if index + 1 == len(articles):
                    pending_article_data = article
                else:
                    yield article

        print(pending_article_data)
        if len(pending_article_data) != 0:
            yield pending_article_data

In [3]:
subword_text_encoder = tfds.features.text.SubwordTextEncoder.build_from_corpus(articles(), 4096)
subword_text_encoder.save_to_file('vocab_4096')

b''


Да видим как би изглеждало обучение с кодираните статии...

In [4]:
import os

import numpy as np
import itertools

In [5]:
BATCH_SIZE = 64
BATCHED_ITEM_LENGTH = 128
BUFFER_SIZE = 256
TYPE=np.uint16

def articles_generator():
    for index, article in enumerate(itertools.islice(articles(), 0, 2000)):
        yield np.array(subword_text_encoder.encode(article + b'\0'), dtype=TYPE)

    # Pad the article count to the batch size
    # We do this to ensure that no data is dropped
    index += 1
    while index % BATCH_SIZE != 0:
        yield np.array([0], dtype=TYPE)
        index += 1

def subbatches():
    dataset = tf.data.Dataset.from_generator(articles_generator, output_types=TYPE)
    dataset = dataset.shuffle(BUFFER_SIZE)
    dataset = dataset.padded_batch(BATCH_SIZE, padded_shapes=([None]), drop_remainder=True)

    for batch in dataset.as_numpy_iterator():
        remaining = batch
        while remaining.shape[1] > 1:
            yield remaining[:, :BATCHED_ITEM_LENGTH]
            remaining = remaining[:, BATCHED_ITEM_LENGTH-1:]

dataset = tf.data.Dataset.from_generator(subbatches, output_types=TYPE, output_shapes=(BATCH_SIZE, None))
dataset = dataset.map(lambda batch: (batch[:, :-1], batch[:, 1:]))

dataset

<MapDataset shapes: ((64, None), (64, None)), types: (tf.uint16, tf.uint16)>

In [6]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    return tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
        tf.keras.layers.LSTM(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size),
    ])

In [7]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

def average_batch_length(true_labels, predictions):
    return tf.shape(true_labels)[1]

model = build_model(vocab_size = subword_text_encoder.vocab_size, embedding_dim=256, rnn_units=1024, batch_size=BATCH_SIZE)
model.compile(optimizer='adam', loss=loss, metrics=[average_batch_length])

In [8]:
checkpoint_dir = './training_checkpoints' # Directory where the checkpoints will be saved
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}") # Name of the checkpoint files

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix, save_weights_only=True)

In [9]:
class ModelStateResetter(tf.keras.callbacks.Callback):
    def __init__(self):
        self.last_total_length = 0

    def on_batch_end(self, batch, logs={}):
        average_batch_length = logs.get('average_batch_length', 0)
        total_length = int(round(average_batch_length * (batch + 1)))
        current_batch_length = total_length - self.last_total_length
        self.last_total_length = total_length
        
        if current_batch_length < BATCHED_ITEM_LENGTH - 1:
            self.model.reset_states()
        
model_state_resetter_callback = ModelStateResetter()

In [10]:
total_epochs = 17

for epoch in range(total_epochs):
    print('Epoch %d/%d' % (epoch + 1, total_epochs))
    model.fit(dataset, callbacks=[checkpoint_callback, model_state_resetter_callback])

Epoch 1/17
4499/4499 [==============================] - 444s 99ms/step - loss: 0.7158 - average_batch_length: 126.5603
Epoch 2/17
4653/4653 [==============================] - 459s 99ms/step - loss: 0.5042 - average_batch_length: 126.5846
Epoch 3/17
4426/4426 [==============================] - 438s 99ms/step - loss: 0.4700 - average_batch_length: 126.5465
Epoch 4/17
4567/4567 [==============================] - 453s 99ms/step - loss: 0.4212 - average_batch_length: 126.5297
Epoch 5/17
4283/4283 [==============================] - 423s 99ms/step - loss: 0.4228 - average_batch_length: 126.5326
Epoch 6/17
4657/4657 [==============================] - 461s 99ms/step - loss: 0.3706 - average_batch_length: 126.5443
Epoch 7/17
4599/4599 [==============================] - 456s 99ms/step - loss: 0.3600 - average_batch_length: 126.5714
Epoch 8/17
4533/4533 [==============================] - 449s 99ms/step - loss: 0.3520 - average_batch_length: 126.5577
Epoch 9/17
4313/4313 [==========================

In [11]:
with open('page_revisions_text', 'rb') as text_file:
    data = text_file.read()

article = data.split(b'\0')[120]
del data

encoded_article = np.array(subword_text_encoder.encode(article + b'\0'), dtype=TYPE)

print('Raw:', len(article))
print('Encded:', len(encoded_article))

Raw: 25541
Encded: 8000


In [12]:
import huffman

def huffman_archive_size(model, text):
    archived_size = 0
    ones = 0
    input_eval = np.array([[0]], dtype=TYPE)

    text_generated = []

    model.reset_states()

    for index, byte in enumerate(text):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0) # remove the batch dimension
  
        probabilities = tf.nn.softmax(predictions[0])
        codebook = huffman.codebook([index, tensor.numpy()] for index, tensor in enumerate(probabilities))

        code = codebook[byte]
        ones += code.count('1')
        archived_size += len(code)

        input_eval = tf.expand_dims([byte], 0)
  
    return ones, archived_size

In [13]:
tf.train.latest_checkpoint(checkpoint_dir)
model = build_model(vocab_size = subword_text_encoder.vocab_size, embedding_dim=256, rnn_units=1024, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [14]:
ones, archived_size = huffman_archive_size(model, encoded_article)
print('\nCompressed length:', archived_size)

compression_ratio = archived_size / (len(encoded_article) * 8)
print('Compression ratio for encoded:', compression_ratio)

compression_ratio = archived_size / (len(article) * 8)
print('Compression ratio for raw:', compression_ratio)

k = (ones / archived_size)
compression_ratio = compression_ratio * (-k * np.log2(k) - (1-k) * np.log2(1-k))
print('Potential compression ratio with arithmetic coding:', compression_ratio)


Compressed length: 32239
Compression ratio for encoded: 0.503734375
Compression ratio for raw: 0.1577806272268118
Potential compression ratio with arithmetic coding: 0.1561948526435945


Да пробваме да тренираме още!

In [15]:
model = build_model(vocab_size = subword_text_encoder.vocab_size, embedding_dim=256, rnn_units=1024, batch_size=BATCH_SIZE)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.compile(optimizer='adam', loss=loss, metrics=[average_batch_length])

In [16]:
total_epochs = 10

for epoch in range(total_epochs):
    print('Epoch %d/%d' % (epoch + 1, total_epochs))
    model.fit(dataset, callbacks=[checkpoint_callback, model_state_resetter_callback])

Epoch 1/10
4582/4582 [==============================] - 456s 100ms/step - loss: 0.2887 - average_batch_length: 126.5884
Epoch 2/10
4354/4354 [==============================] - 434s 100ms/step - loss: 0.3009 - average_batch_length: 126.5783
Epoch 3/10
4464/4464 [==============================] - 445s 100ms/step - loss: 0.2917 - average_batch_length: 126.5197
Epoch 4/10


4349/4349 [==============================] - 433s 100ms/step - loss: 0.2980 - average_batch_length: 126.5532
Epoch 5/10
4571/4571 [==============================] - 453s 99ms/step - loss: 0.2823 - average_batch_length: 126.5874
Epoch 6/10
4341/4341 [==============================] - 428s 99ms/step - loss: 0.2972 - average_batch_length: 126.5185
Epoch 7/10
4563/4563 [==============================] - 450s 99ms/step - loss: 0.2821 - average_batch_length: 126.5268
Epoch 8/10
4384/4384 [==============================] - 432s 99ms/step - loss: 0.2938 - average_batch_length: 126.5167
Epoch 9/10
4482/4482 [==============================] - 441s 98ms/step - loss: 0.2868 - average_batch_length: 126.4697
Epoch 10/10
4349/4349 [==============================] - 429s 99ms/step - loss: 0.2965 - average_batch_length: 126.5256


In [17]:
total_epochs = 10

for epoch in range(total_epochs):
    print('Epoch %d/%d' % (epoch + 1, total_epochs))
    model.fit(dataset, callbacks=[checkpoint_callback, model_state_resetter_callback])

Epoch 1/10
4628/4628 [==============================] - 453s 98ms/step - loss: 0.2786 - average_batch_length: 126.5445
Epoch 2/10
4298/4298 [==============================] - 423s 98ms/step - loss: 0.2998 - average_batch_length: 126.5407
Epoch 3/10
4445/4445 [==============================] - 437s 98ms/step - loss: 0.2900 - average_batch_length: 126.5442
Epoch 4/10
4577/4577 [==============================] - 452s 99ms/step - loss: 0.2869 - average_batch_length: 126.5820
Epoch 5/10
4290/4290 [==============================] - 429s 100ms/step - loss: 0.3024 - average_batch_length: 126.5189
Epoch 6/10
4345/4345 [==============================] - 435s 100ms/step - loss: 0.2996 - average_batch_length: 126.5496
Epoch 7/10
4465/4465 [==============================] - 447s 100ms/step - loss: 0.2917 - average_batch_length: 126.5350
Epoch 8/10
4465/4465 [==============================] - 446s 100ms/step - loss: 0.2946 - average_batch_length: 126.5534
Epoch 9/10
4348/4348 [======================

Да пробваме да намалим скоростта на обучение след определен брой итерации:

In [23]:
total_epochs = 5

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', verbose=1)

for epoch in range(total_epochs):
    print('Epoch %d/%d' % (epoch + 1, total_epochs))
    model.fit(dataset, callbacks=[checkpoint_callback, model_state_resetter_callback, reduce_lr])

Epoch 1/5
4632/4632 [==============================] - 455s 98ms/step - loss: 0.2922 - average_batch_length: 126.5607
Epoch 2/5
4544/4544 [==============================] - 448s 99ms/step - loss: 0.2994 - average_batch_length: 126.5376
Epoch 3/5
4415/4415 [==============================] - 439s 99ms/step - loss: 0.3065 - average_batch_length: 126.5925
Epoch 4/5
4470/4470 [==============================] - 447s 100ms/step - loss: 0.2998 - average_batch_length: 126.5011
Epoch 5/5
4389/4389 [==============================] - 435s 99ms/step - loss: 0.3063 - average_batch_length: 126.5616


Изглежда няма да постигнем нещо смислено с повече учене